### Assignment

In [ ]:
##ps.pankajsakharkar@gmail.com

In [ ]:
#Load data first

import pandas as pd
# Load data from the first sheet
df = pd.read_excel("Artificial_Data.xlsx", sheet_name="Sheet1")

In [ ]:
## Function to analyze sentiment
from transformers import pipeline
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path,token="")

def get_sentiment_score(text): 
    """This function will return sentiment score of text between 0 -1 """
    sentiment = sentiment_task(text)
    return sentiment[0]["score"]


e:\ps_project_task\study\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [ ]:
#sample Data Analysis

In [4]:
from sklearn.preprocessing import LabelEncoder

rating_encoder = LabelEncoder()
df["Rating_encoded"] = rating_encoder.fit_transform(df["Rating"])

df["sentiment"] = df['string_values'].apply(get_sentiment_score)

In [5]:
import re
from nltk.corpus import stopwords
from keybert import KeyBERT
kw_model = KeyBERT()

# Text cleaning function
def get_keywords(text):
    """Function to return impo keywords."""
    text = text.lower()  # Lowercase
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = kw_model.extract_keywords(text)
    tokens = [i[0] for i in text] # keyword tokens
    tokens = [word for word in tokens if word not in stopwords.words("english")]  # Remove stopwords
    return " ".join(tokens) # return in string form

In [6]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical ratings
rating_encoder = LabelEncoder()
df["Rating_encoded"] = rating_encoder.fit_transform(df["Rating"])

# Apply key words extraction
df["keywords"] = df["string_values"].astype(str).apply(get_keywords)



In [8]:
## Prepared features of text data using TF-IDF techniques

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text into TF-IDF features
tfidf_vectorizer = TfidfVectorizer(max_features=100)
tfidf_features = tfidf_vectorizer.fit_transform(df["keywords"])

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenate with original dataset
df_combined = pd.concat([df, tfidf_df], axis=1)

df_combined.head()


,RATING_TYPE,Rating,FundaIndxint,FundaIndxsga,monthvwretd,monthvwretx,monthewretd,monthewretx,monthsprtrn,monthspindx,...,markets,outlook,positive,reduction,revenue,strategic,supply,taxes,volatility,year
0,Fitch,A+,0.150659,-0.420047,-0.846934,0.560226,0.579303,-0.578314,-0.225006,-0.415867,...,0.0,0.0,0.0,0.0,0.000000,0.447214,0.447214,0.0,0.0,0.0
1,Moody's,BB,-0.015715,0.139204,-0.045250,-0.912128,0.497292,-0.910558,1.207592,-0.104050,...,0.0,0.0,0.0,0.0,0.447214,0.000000,0.000000,0.0,0.0,0.0
2,Fitch,BB,-1.330074,0.001814,-2.564342,0.997115,0.762847,-0.870865,0.230487,0.515273,...,0.0,0.0,0.0,0.0,0.447214,0.000000,0.000000,0.0,0.0,0.0
3,Fitch,BBB,-0.025288,0.755935,1.142726,-0.235319,0.637967,-1.611961,0.432309,1.194182,...,0.0,0.0,0.0,0.0,0.447214,0.000000,0.000000,0.0,0.0,0.0
4,Moody's,AA,1.343168,1.933543,-1.896747,0.467780,-0.442432,1.349705,0.541002,0.096545,...,0.0,0.0,0.0,0.0,0.447214,0.000000,0.000000,0.0,0.0,0.0


In [10]:
## Data spliting

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


X_structured = df_combined.drop(columns=["Rating", "RATING_TYPE", "string_values", "keywords", "Rating_encoded"])
X_text = tfidf_df                   # Text-only features
y = df_combined["Rating_encoded"]   # Target variable

# Spliting data 
X_train_struct, X_test_struct, y_train, y_test = train_test_split(X_structured, y, test_size=0.2, random_state=42)
X_train_text, X_test_text, _, _ = train_test_split(X_text, y, test_size=0.2, random_state=42)

In [ ]:
## Now I wil build three models -
# 1 - without string data (structured data only)
# 2 - with string type TFIDF data
# 3 - with combined data 

# then build models and check predictions and metrics (classification report, model accuracy)

#### Decision Tree model

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Define target and feature sets
X_structured = df_combined.drop(columns=["Rating", "RATING_TYPE", "string_values", "keywords", "Rating_encoded"])
X_text = tfidf_df  # Text-only features
y = df_combined["Rating_encoded"]  # Target variable

# Split data 
X_train_struct, X_test_struct, y_train, y_test = train_test_split(X_structured, y, test_size=0.2, random_state=42)
X_train_text, X_test_text, _, _ = train_test_split(X_text, y, test_size=0.2, random_state=42)

# Train models
model_structured = DecisionTreeClassifier( random_state=42)
model_text = DecisionTreeClassifier( random_state=42)
model_combined = DecisionTreeClassifier( random_state=42)

model_structured.fit(X_train_struct, y_train)
model_text.fit(X_train_text, y_train)
model_combined.fit(pd.concat([X_train_struct, X_train_text], axis=1), y_train)

# Predictions
y_pred_struct = model_structured.predict(X_test_struct)
y_pred_text = model_text.predict(X_test_text)
y_pred_combined = model_combined.predict(pd.concat([X_test_struct, X_test_text], axis=1))

# Evaluate performance
report_struct = classification_report(y_test, y_pred_struct)
report_text = classification_report(y_test, y_pred_text)
report_combined = classification_report(y_test, y_pred_combined)




e:\ps_project_task\study\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ps_project_task\study\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ps_project_task\study\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ps_project_task\study\

In [15]:
## Evaluation 

In [16]:
print(report_struct)

              precision    recall  f1-score   support

           0       0.33      0.67      0.44         3
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         2

    accuracy                           0.10        20
   macro avg       0.04      0.08      0.06        20
weighted avg       0.05      0.10      0.07        20



In [17]:
print(report_text)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.10      0.25      0.14         4
           7       0.00      0.00      0.00         2

    accuracy                           0.05        20
   macro avg       0.01      0.03      0.02        20
weighted avg       0.02      0.05      0.03        20



In [18]:
print(report_combined)

              precision    recall  f1-score   support

           0       0.50      0.67      0.57         3
           1       0.50      0.33      0.40         3
           2       0.00      0.00      0.00         2
           3       0.25      0.25      0.25         4
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         2

    accuracy                           0.20        20
   macro avg       0.16      0.16      0.15        20
weighted avg       0.20      0.20      0.20        20



#### Conclusion for decision Tree model : 
- model-3 accuracy is 20%
- here data volumne is very low for model building.
- If we could have mode text data , we can extract more important keywords and do sentiment analysis techniques.
- model 3 accuracy is high as comapre to other models , this model is combination of sentiment score feature, extracted keywords featurs with original structured data.


#### Random Forest model

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Define target and feature sets
X_structured = df_combined.drop(columns=["Rating", "RATING_TYPE", "string_values", "keywords", "Rating_encoded"])
X_text = tfidf_df  # Text-only features
y = df_combined["Rating_encoded"]  # Target variable

# Split data 
X_train_struct, X_test_struct, y_train, y_test = train_test_split(X_structured, y, test_size=0.2, random_state=42)
X_train_text, X_test_text, _, _ = train_test_split(X_text, y, test_size=0.2, random_state=42)

# Train models
model_structured = RandomForestClassifier(n_estimators=50, random_state=42)
model_text = RandomForestClassifier(n_estimators=50, random_state=42)
model_combined = RandomForestClassifier(n_estimators=50, random_state=42)

model_structured.fit(X_train_struct, y_train)
model_text.fit(X_train_text, y_train)
model_combined.fit(pd.concat([X_train_struct, X_train_text], axis=1), y_train)

# Predictions
y_pred_struct = model_structured.predict(X_test_struct)
y_pred_text = model_text.predict(X_test_text)
y_pred_combined = model_combined.predict(pd.concat([X_test_struct, X_test_text], axis=1))

# Evaluate performance
report_struct = classification_report(y_test, y_pred_struct)
report_text = classification_report(y_test, y_pred_text)
report_combined = classification_report(y_test, y_pred_combined)




e:\ps_project_task\study\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ps_project_task\study\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ps_project_task\study\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ps_project_task\study\

In [ ]:
## Evaluation 

In [21]:
print(report_struct)

              precision    recall  f1-score   support

           0       1.00      0.33      0.50         3
           1       0.12      0.33      0.18         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         2

    accuracy                           0.10        20
   macro avg       0.14      0.08      0.09        20
weighted avg       0.17      0.10      0.10        20



In [22]:
print(report_text)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.10      0.25      0.14         4
           7       0.00      0.00      0.00         2

    accuracy                           0.05        20
   macro avg       0.01      0.03      0.02        20
weighted avg       0.02      0.05      0.03        20



In [23]:
print(report_combined)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.12      0.25      0.17         4
           7       0.00      0.00      0.00         2

    accuracy                           0.05        20
   macro avg       0.02      0.03      0.02        20
weighted avg       0.03      0.05      0.03        20



#### Conclusion for Random forest model : 
- model-1 accuracy is 10%
- here data volumne is very low for model building.
- If we could have mode text data , we can extract more important keywords and do sentiment analysis techniques.
- model 3 accuracy is high as comapre to other models , this model is combination of sentiment score feature, extracted keywords featurs with original structured data.